# Tutorial 02 - Elastic Block

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import gmsh
import multiphenicsx.io
import multiphenicsx.mesh
import numpy as np
import petsc4py
import ufl

In [ ]:
import rbnicsx.backends

## 1. Mesh generation

In [ ]:
def generate_rectangular_subdomains(
    x: typing.List[float], y: typing.List[float], mesh_size: float
) -> typing.Tuple[int]:
    """Add points and lines that define rectangular subdomains with the provided coordinates."""
    points = [[gmsh.model.geo.addPoint(x_, y_, 0.0, mesh_size) for y_ in y] for x_ in x]
    horizontal_lines = [
        [gmsh.model.geo.addLine(points[i][j], points[i + 1][j]) for j in range(len(points[i]))]
        for i in range(len(points) - 1)]
    vertical_lines = [
        [gmsh.model.geo.addLine(points[i][j], points[i][j + 1]) for j in range(len(points[i]) - 1)]
        for i in range(len(points))]
    curve_loops = [
        [gmsh.model.geo.addCurveLoop([
            horizontal_lines[i][j], vertical_lines[i + 1][j],
            - horizontal_lines[i][j + 1], - vertical_lines[i][j]]) for j in range(len(points[i]) - 1)]
        for i in range(len(points) - 1)]
    boundaries = [
        [{
            "horizontal": (
                horizontal_lines[i][j] if j == 0 else
                horizontal_lines[i][j + 1] if j == len(points[i]) - 2 else None),
            "vertical": (
                vertical_lines[i][j] if i == 0 else
                vertical_lines[i + 1][j] if i == len(points) - 2 else None)
        } for j in range(len(points[i]) - 1)]
        for i in range(len(points) - 1)]
    subdomains = [
        [gmsh.model.geo.addPlaneSurface([curve_loops[i][j]]) for j in range(len(points[i]) - 1)]
        for i in range(len(points) - 1)]
    return subdomains, boundaries

In [ ]:
mesh_size = 5e-2

In [ ]:
gmsh.initialize()
gmsh.model.add("elastic_block")

In [ ]:
gmsh_subdomains, gmsh_boundaries = generate_rectangular_subdomains(
    [i / 3. for i in range(4)], [j / 3. for j in range(4)], mesh_size)

In [ ]:
gmsh.model.geo.synchronize()
for i in range(3):
    for j in range(3):
        gmsh.model.addPhysicalGroup(2, [gmsh_subdomains[i][j]], i + 3 * j + 1)
gmsh.model.addPhysicalGroup(1, [gmsh_boundaries[i][0]["horizontal"] for i in range(3)], 1)
[gmsh.model.addPhysicalGroup(1, [gmsh_boundaries[2][j]["vertical"]], j + 2) for j in range(3)]
gmsh.model.addPhysicalGroup(1, [gmsh_boundaries[i][2]["horizontal"] for i in range(3)], 5)
gmsh.model.addPhysicalGroup(1, [gmsh_boundaries[0][j]["vertical"] for j in range(3)], 6)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

## 2. Problem definition

In [ ]:
class Problem(object):
    """Define a linear problem, and solve it with KSP."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", 1))
        self._V = V
        # Define trial and test functions
        u = ufl.TrialFunction(V)
        v = ufl.TestFunction(V)
        # Define measures for integration of forms
        dx = ufl.Measure("dx")(subdomain_data=subdomains)
        ds = ufl.Measure("ds")(subdomain_data=boundaries)
        # Define symbolic parameters for use in UFL forms
        mu_symb = rbnicsx.backends.SymbolicParameters(mesh, shape=(11, ))
        self._mu_symb = mu_symb
        # Define bilinear form of the problem
        a = sum([(mu_symb[p] if p < 8 else 1.0) * self._elasticity(u, v) * dx(p + 1) for p in range(9)])
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        # Define linear form of the problem
        f = sum([ufl.inner(mu_symb[p], v[0]) * ds(p - 6) for p in range(8, 11)])
        self._f = f
        self._f_cpp = dolfinx.fem.form(f)
        # Define boundary conditions for the problem
        zero_vector = np.zeros(mesh.topology.dim, petsc4py.PETSc.ScalarType)
        facets_left = boundaries.indices[boundaries.values == 6]
        bdofs_V_left = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_left)
        bcs = [dolfinx.fem.dirichletbc(zero_vector, bdofs_V_left, V)]
        self._bcs = bcs

    @staticmethod
    def _elasticity(u: ufl.Argument, v: ufl.Argument) -> ufl.core.expr.Expr:
        """Compute the elasticity bilinear form."""
        E = 1.0
        nu = 0.3
        lambda_1 = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))
        lambda_2 = E / (2.0 * (1.0 + nu))
        return (
            2.0 * lambda_2 * ufl.inner(ufl.sym(ufl.grad(u)), ufl.sym(ufl.grad(v)))
            + lambda_1 * ufl.inner(ufl.tr(ufl.sym(ufl.grad(u))), ufl.tr(ufl.sym(ufl.grad(v)))))

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpace:
        """Return the function space of the problem."""
        return self._V

    @property
    def bilinear_form(self) -> ufl.Form:
        """Return the bilinear form of the problem."""
        return self._a

    @property
    def linear_form(self) -> ufl.Form:
        """Return the linear form of the problem."""
        return self._f

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBCMetaClass]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    def _assemble_matrix(self) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side matrix."""
        A = dolfinx.fem.petsc.assemble_matrix(self._a_cpp, bcs=self._bcs)
        A.assemble()
        return A

    def _assemble_vector(self) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side vector."""
        F = dolfinx.fem.petsc.assemble_vector(self._f_cpp)
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        dolfinx.fem.petsc.set_bc(F, self._bcs)
        return F

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Assign the provided parameters value and solve the problem."""
        self._mu_symb.value[:] = mu
        return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the linear problem with KSP."""
        A = self._assemble_matrix()
        F = self._assemble_vector()
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        solution = dolfinx.fem.Function(self._V)
        ksp.solve(F, solution.vector)
        solution.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        return solution

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.hstack((np.ones(8), np.array([1.0, -1.0, -1.0])))
solution = problem.solve(mu_solve)

In [ ]:
multiphenicsx.io.plot_vector_field(solution, "high fidelity solution")